In [1]:
import numpy as np
import pandas as pd

In [2]:
# import the  data

app_downloads = pd.read_csv('metrocar_data/app_downloads.csv')
reviews = pd.read_csv('metrocar_data/reviews.csv')
ride_requests = pd.read_csv('metrocar_data/ride_requests.csv')
signups = pd.read_csv('metrocar_data/signups.csv')
transactions = pd.read_csv('metrocar_data/transactions.csv')

/var/folders/1m/hvnyjbcx2rxfb9d__rb_m7h00000gp/T/ipykernel_27688/650494655.py:5: DtypeWarning: Columns (4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  ride_requests = pd.read_csv('metrocar_data/ride_requests.csv')


In [26]:
df = app_downloads.merge(signups, left_on='app_download_key', right_on='session_id', how='left' )\
                  .merge(ride_requests, on='user_id', how='inner')\
                  .merge(transactions, on='ride_id', how='inner')\
                  .merge(reviews, on=['user_id','ride_id','driver_id'], how='inner')

# downloads_df.merge(signups_df, left_on='app_download_key', right_on='session_id', how='inner')

# df.to_csv('full_metrocar_data.csv', sep=',', index=False, encoding='utf-8')

In [24]:

df_notnull = df[df['dropoff_ts'].notnull()]
grouped_count = df_notnull.groupby('user_id').size()
grouped_count

user_id
100002.0    37
100004.0    32
100007.0    28
100008.0    42
100010.0    44
            ..
117614.0    42
117615.0    33
117617.0    30
117621.0    37
117622.0    26
Length: 6233, dtype: int64

In [4]:
df.columns

Index(['app_download_key', 'platform', 'download_ts', 'user_id', 'session_id',
       'signup_ts', 'age_range', 'ride_id', 'driver_id', 'request_ts',
       'accept_ts', 'pickup_location', 'dropoff_location', 'pickup_ts',
       'dropoff_ts', 'cancel_ts', 'transaction_id', 'purchase_amount_usd',
       'charge_status', 'transaction_ts', 'review_id', 'rating', 'review'],
      dtype='object')

In [6]:
# columns to include in df

df_user = df.copy()

df_user = df_user[['user_id', 'age_range', 'platform', 'download_ts', 
                   'signup_ts', 'request_ts', 'accept_ts', 'pickup_ts', 
                   'dropoff_ts', 'cancel_ts', 'transaction_ts', 'charge_status', 
                   'purchase_amount_usd','review_id', 'rating']]

# convert columns to booleans

columns_to_convert = ['download_ts', 'signup_ts', 'request_ts', 'accept_ts', 
                      'pickup_ts', 'dropoff_ts', 'cancel_ts', 'transaction_ts', 
                      'review_id']

for column in columns_to_convert:
    new_column_name = column.replace('_ts', '')
    df_user[column] = df_user[column].notna().astype(int)
    df_user = df_user.rename(columns={column: new_column_name})



# convert the charge status to a boolean

df_user['charge_status'] = df_user['charge_status'].map({'Approved': 1, 
                                                         'Decline': 0})

# group by user_id

# assigin columns as categorical or numerical data

categorical = ['age_range', 'platform']
numerical = ['download','signup', 'request', 'accept', 'pickup', 
            'dropoff',	'cancel', 'transaction', 'charge_status', 
            'purchase_amount_usd', 'review_id', 'rating']

# set aggregations for categorical and numrical columns
aggregations = {}
for cat_col in categorical:
    aggregations[cat_col] = 'first'

for num_col in numerical:
    aggregations[num_col] = 'mean'


# group by user id
# reset_index to avoid user_id being used as an index

df_user = df_user.groupby('user_id').agg(aggregations).reset_index()

In [7]:
df_user.head()

,user_id,age_range,platform,download,signup,request,accept,pickup,dropoff,cancel,transaction,charge_status,purchase_amount_usd,review_id,rating
0,100000.0,None,android,1.0,1.0,1.0,0.269231,0.0,0.0,1.0,0.0,NaN,NaN,0.0,NaN
1,100001.0,25-34,ios,1.0,1.0,1.0,0.260870,0.0,0.0,1.0,0.0,NaN,NaN,0.0,NaN
2,100002.0,35-44,ios,1.0,1.0,1.0,1.000000,1.0,1.0,0.0,1.0,0.891892,20.531081,1.0,3.459459
3,100003.0,25-34,ios,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN
4,100004.0,35-44,ios,1.0,1.0,1.0,1.000000,1.0,1.0,0.0,1.0,0.968750,20.927500,1.0,2.875000


In [8]:
# columns to include in df

df_driver = df.copy()

df_driver = df_driver[['driver_id', 'request_ts', 'accept_ts', 'pickup_ts', 
                       'dropoff_ts', 'cancel_ts', 'transaction_ts', 
                       'charge_status', 'purchase_amount_usd','review_id', 
                       'rating']]

# convert columns to booleans

columns_to_convert = ['request_ts', 'accept_ts', 'pickup_ts', 'dropoff_ts', 
                      'cancel_ts', 'transaction_ts', 'review_id']

for column in columns_to_convert:
    new_column_name = column.replace('_ts', '')
    df_driver[column] = df_driver[column].notna().astype(int)
    df_driver = df_driver.rename(columns={column: new_column_name})



# convert the charge status to a boolean

df_driver['charge_status'] = df_driver['charge_status'].map({'Approved': 1, 'Decline': 0})

# group by user_id

# assigin columns as categorical or numerical data


numerical = ['request', 'accept', 'pickup', 
            'dropoff',	'cancel', 'transaction', 'charge_status', 
            'purchase_amount_usd', 'review_id', 'rating']

# set aggregations for categorical and numrical columns
aggregations = {}


for num_col in numerical:
    aggregations[num_col] = 'mean'


# group by user id
# reset_index to avoid user_id being used as an index

df_driver = df_driver.groupby('driver_id').agg(aggregations).reset_index()

In [10]:
df_driver.head()

,driver_id,request,accept,pickup,dropoff,cancel,transaction,charge_status,purchase_amount_usd,review_id,rating
0,100000.0,1.0,1.0,0.904762,0.904762,0.095238,0.904762,1.000000,19.858421,0.761905,2.687500
1,100001.0,1.0,1.0,0.950000,0.950000,0.050000,0.950000,0.947368,22.499474,0.700000,3.428571
2,100002.0,1.0,1.0,1.000000,1.000000,0.000000,1.000000,0.875000,18.121875,0.750000,3.500000
3,100003.0,1.0,1.0,0.777778,0.777778,0.222222,0.777778,0.857143,24.595714,0.444444,3.750000
4,100004.0,1.0,1.0,1.000000,1.000000,0.000000,1.000000,1.000000,20.845000,0.700000,3.000000


In [11]:
# columns to include in df

df_combined = df.copy()

df_combined = df_combined[['user_id', 'driver_id', 'age_range', 'platform', 'download_ts', 
                   'signup_ts', 'request_ts', 'accept_ts', 'pickup_ts', 
                   'dropoff_ts', 'cancel_ts', 'transaction_ts', 'charge_status', 
                   'purchase_amount_usd','review_id', 'rating']]

# convert columns to booleans

columns_to_convert = ['download_ts', 'signup_ts', 'request_ts', 'accept_ts', 
                      'pickup_ts', 'dropoff_ts', 'cancel_ts', 'transaction_ts', 
                      'review_id']

for column in columns_to_convert:
    new_column_name = column.replace('_ts', '')
    df_combined[column] = df_combined[column].notna().astype(int)
    df_combined = df_combined.rename(columns={column: new_column_name})


# convert the charge status to a boolean

df_combined['charge_status'] = df_combined['charge_status'].map({'Approved': 1, 
                                                         'Decline': 0})


In [16]:
df_combined.head(50)

,user_id,driver_id,age_range,platform,download,signup,request,accept,pickup,dropoff,cancel,transaction,charge_status,purchase_amount_usd,review_id,rating
0,106328.0,NaN,None,android,1,1,0,0,0,0,0,0,NaN,NaN,0,NaN
1,NaN,NaN,NaN,android,1,0,0,0,0,0,0,0,NaN,NaN,0,NaN
2,100497.0,101863.0,45-54,android,1,1,1,1,1,1,0,1,1.0,13.05,1,4.0
3,100497.0,102362.0,45-54,android,1,1,1,1,1,1,0,1,1.0,25.10,1,1.0
4,100497.0,103914.0,45-54,android,1,1,1,1,1,1,0,1,1.0,18.38,1,1.0
5,100497.0,101250.0,45-54,android,1,1,1,1,1,1,0,1,1.0,27.20,1,5.0
6,100497.0,114633.0,45-54,android,1,1,1,1,1,1,0,1,1.0,22.49,1,4.0
7,100497.0,108463.0,45-54,android,1,1,1,1,1,1,0,1,1.0,11.39,1,1.0
8,100497.0,106186.0,45-54,android,1,1,1,1,1,1,0,1,1.0,24.33,1,1.0
9,100497.0,101087.0,45-54,android,1,1,1,1,1,1,0,1,1.0,12.95,1,4.0


In [18]:
# df_combined.to_csv('combined_metrocar_data.csv', sep=',', index=False, encoding='utf-8')

In [15]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396679 entries, 0 to 396678
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              390694 non-null  float64
 1   driver_id            248379 non-null  float64
 2   age_range            390694 non-null  object 
 3   platform             396679 non-null  object 
 4   download             396679 non-null  int64  
 5   signup               396679 non-null  int64  
 6   request              396679 non-null  int64  
 7   accept               396679 non-null  int64  
 8   pickup               396679 non-null  int64  
 9   dropoff              396679 non-null  int64  
 10  cancel               396679 non-null  int64  
 11  transaction          396679 non-null  int64  
 12  charge_status        223652 non-null  float64
 13  purchase_amount_usd  223652 non-null  float64
 14  review_id            396679 non-null  int64  
 15  rating           

In [13]:
df_combined['signup'].value_counts()

1    390694
0      5985
Name: signup, dtype: int64

In [14]:
transactions['charge_status'].value_counts()

Approved    212628
Decline      11024
Name: charge_status, dtype: int64